In [ ]:
# Install dependencies
!pip install pandas scikit-learn transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

In [12]:
# Step 1: Load Data
# Load training data
csv_path = 'trainingbook_utf8.csv'
training_data = pd.read_csv(csv_path)  # Specify column names and skip header if present

# Load testing data (interview responses)
txt_path = 'interview_responses.txt'
with open(txt_path, 'r', encoding='utf-8') as file:
    test_data = [line.strip() for line in file if line.strip()]

In [13]:
# Step 2: Preprocess Training Data
# Mapping biases to numeric labels
bias_mapping = {
    "Confirmation Bias": 0, "Backfire Effect": 1, "Naïve Realism": 2,
    "Out-Group Homogeneity Effect": 3, "Anchoring Bias": 4, "Authority Bias": 5,
    "Belief Perseverance": 6, "Status Quo Bias": 7, "System Justification": 8
}


In [15]:
# Map biases to numeric labels
training_data['label'] = training_data['bias'].map(bias_mapping)

# Drop rows with invalid or unmapped labels
training_data = training_data.dropna(subset=['label'])

# Ensure labels are integers
training_data['label'] = training_data['label'].astype(int)

# Split data into train and evaluation sets
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    training_data['text'], training_data['label'], test_size=0.2, random_state=42
)

model_name = "bert-base-uncased"  # Example: using bert-base-uncased

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Tokenize the training and evaluation data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512, return_tensors="pt")
eval_encodings = tokenizer(list(eval_texts), truncation=True, padding=True, max_length=512, return_tensors="pt")

# Prepare datasets
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': torch.tensor(train_labels.tolist(), dtype=torch.long)  # Convert labels to torch tensor
})

eval_dataset = Dataset.from_dict({
    'input_ids': eval_encodings['input_ids'],
    'attention_mask': eval_encodings['attention_mask'],
    'labels': torch.tensor(eval_labels.tolist(), dtype=torch.long)  # Convert labels to torch tensor
})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
print(train_labels.unique())  # Check for unexpected or invalid values


[0 4 3 5 6 2 7]


In [17]:
# Load the BERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(bias_mapping))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',  # Updated argument
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch",
    load_best_model_at_end=True
)

from sklearn.metrics import accuracy_score, f1_score

# Define a compute_metrics function
def compute_metrics(pred):
    logits, labels = pred
    predictions = logits.argmax(axis=-1)  # Get the predicted class
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")  # Use weighted average for F1
    return {"accuracy": accuracy, "f1": f1}

# Pass the compute_metrics function to the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics  # Add this line
)

# Train the model
trainer.train()

# Tokenize testing data
test_encodings = tokenizer(test_data, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Prepare the testing dataset
test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask']
})

# Predict on testing data
test_predictions = trainer.predict(test_dataset)
test_bias_labels = test_predictions.predictions.argmax(axis=1)

# Map predicted labels back to biases
reverse_bias_mapping = {v: k for k, v in bias_mapping.items()}
test_bias_names = [reverse_bias_mapping[label] for label in test_bias_labels]

# Combine results into a DataFrame
test_results = pd.DataFrame({
    'text': test_data,
    'bias': test_bias_names
})

# Save results to a CSV file
test_results.to_csv('bert_test_results.csv', index=False)

# Display a preview of the results
print(test_results.head())

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.252427,0.714286,0.690371
2,No log,0.765697,0.907143,0.906663
3,No log,0.636609,0.942857,0.942310


                                                text            bias
0  ﻿1\tJe n’ai pas peur de dire que d’un point de...  Authority Bias
1  2\tC’est un beau mot ! Radical, pour moi, ça v...  Authority Bias
2  3\tPour moi radical, du moins étymologiquement...  Authority Bias
3  4\t[...] ‘radical’ c’est ce qui s’attaque à la...  Authority Bias
4  5\tJe trouve que radical, c’est prendre le mal...   Naïve Realism


In [19]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)  # Displays evaluation loss, accuracy, and F1-score


{'eval_loss': 0.6366094350814819, 'eval_accuracy': 0.9428571428571428, 'eval_f1': 0.9423099472347077, 'eval_runtime': 1.124, 'eval_samples_per_second': 124.55, 'eval_steps_per_second': 16.014, 'epoch': 3.0}
